In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import time
from pprint import pprint
import re

nr_ur_rbi_fixer_re = '(\((NR|UR|RBI)\))+'

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

events_by_season_dir = '../input/baseball-events-from-retrosheetorg/events_by_season/'
metadata_dir = '../input/baseball-events-from-retrosheetorg/'


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

thisyear = 1919
#first_eventID is x+1 for x=.... 1915: 0, 1916: 104242, 1917: 172315, 1918: 228694, 1919: 280585
first_eventID = 280585 + 1    # 1918

if (thisyear == 1915):
    league_list = ['F', 'A', 'N']
else:
    league_list = ['A', 'N']

new_columns = ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b', 'runner_3b']

final_column_order = ['gameID',
 'event_in_game',
 'inning',
 'visitor_or_home',
  'vis_score',
 'home_score',
 'outs',
 'runner_1b',
 'runner_2b',
 'runner_3b',
 'batterID',
 'batter_hand',
 'pitch_count',
 'pitch_sequence',
 'play_orig',
 'pitcherID',
 'pitcher_hand',
 'field_C_playerID',
 'field_1B_playerID',
 'field_2B_playerID',
 'field_3B_playerID',
 'field_SS_playerID',
 'field_LF_playerID',
 'field_CF_playerID',
 'field_RF_playerID',
 'bat_DH_playerID',
 'pinch_runner_playerID',
 'pinch_runner_replaced_playerID',
 'comments',
 'badj',
 'padj',
 'theyear',
 'visteam',
 'hometeam',
 'theplay',
 'baserunning',
 'play_orig']

early_end_game_list = ['PHI191507210', 'CHN191607180', 'NY1191609080', 'NY1191609182', 'PHA191905050']
# early_end_game_list += ['CHA192104210']
# early_end_game_list += ['CIN202106300', 'PIT202106100', 'SLN202110030']

special_case_events = {}
special_case_events.update({'CHA191505010': [18], 'NY1191509290': [22], 'PHI191507140': [43], 'PHI191509092': [40]})
special_case_events.update({'BOS191609280': [35], 'PHA191605170': [22], 'BRO191608040': [16], 'NY1191606210': [8], 'PIT191605130': [57]})
special_case_events.update({'PHA191708111': [37], 'PHI191710022': [25], 'PIT191706090': [3], 'PIT191706210': [8]})

special_case_events.update({'CHA191806200': [67], 'PHA191807120': [56]})

special_case_events.update({'CHA191907100': [52], 'DET191906021': [28]})


# # special_case_events.update({'NY1192008151': [66], 'PHI192005060': [5], 'PHI192009100': [53]})

# # special_case_events.update({'CHN192108032': [63], 'CIN192104180': [31], 'CIN192104240': [55], 'CIN192106110': [26], 'NY1192105040': [39], 'NY1192105240': [22], 'NY1192105302': [85], 'NY1192106210': [57]})
# # special_case_events.update({'ARI202107310': [16], 'ARI202108300': [5], 'ATL202106201': [32], 'CHN202107080': [13], 'CIN202104160': [32]})

def ordinaltg(n):
    return str(n) + {1: 'st', 2: 'nd', 3: 'rd'}.get(4 if 10 <= n % 100 < 20 else n % 10, "th")
g = 0

print('ok')
#special_case_events

ok


In [28]:
s = time.time()
print('Adding game context data for games in {0:.0f}...'.format(thisyear))
print('\n')
print('reading rosters...')
rosters_df = pandas.read_csv(metadata_dir+'rosters.csv', low_memory=False, index_col='rosterID')

print('reading games...')
games_df = pandas.read_csv(metadata_dir+'games.csv', low_memory=False, index_col='gameID', encoding='utf-8', converters={'visitor_linescore': lambda x: str(x), 'home_linescore': lambda x: str(x)})

print('\tfixing linescores...')
# games_df.loc[games_df['visitor_linescore'] == '"nan"', 'visitor_linescore'] = np.nan
# games_df.loc[games_df['home_linescore'] == '"nan"', 'home_linescore'] = np.nan
# games_df.loc[:, 'visitor_linescore'] = games_df['visitor_linescore'].apply(lambda x: str(str(x)[1:-1]))
# games_df.loc[:, 'home_linescore'] = games_df['home_linescore'].apply(lambda x: str(str(x)[1:-1]))
games_df = games_df.assign(has_linescores = True)
games_df.loc[(games_df['visitor_linescore'].isnull()) | (games_df['home_linescore'].isnull()), 'has_linescores'] = False

print('\tchecking for DH...')
has_dh = True
if (thisyear < 1973):
    has_dh = False

print('\tgetting fielder columns...')
fielder_columns = ['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID']
fielder_columns += ['field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID'] 
if (has_dh):
    fielder_columns += ['bat_DH_playerID']


print('selecting games and roster rows for {0:.0f}...'.format(thisyear))
games_df.loc[:, 'date'] = pandas.to_datetime(games_df['date'], format='%Y%m%d', errors='coerce')
games_df = games_df[games_df['date'].apply(lambda x: x.year == thisyear)]
rosters_df = rosters_df[rosters_df['theyear'] == thisyear]

events_df = pandas.DataFrame()
eventfile_list = []
for thisleague in league_list:
    eventfile_list.append('{0}{1:.0f}/{1:.0f}{2:}/events_{1:.0f}{2:}.csv'.format(events_by_season_dir, thisyear, thisleague.lower()))

# print('Reading these files:')
# pprint(eventfile_list)
# print('\n')

for thisfile in eventfile_list:
    events_i_df = pandas.read_csv(thisfile, encoding='utf-8', index_col='eventID', low_memory=False)#, dtype={'pitch_count': 'str'})
    events_df = pandas.concat((events_df, events_i_df), axis=0)

events_df = events_df.reset_index(drop=True)
events_df.index.name = 'eventID'

events_df = events_df.reset_index()
print('Setting first eventID to be {0:,.0f}...'.format(first_eventID))
events_df.loc[:, 'eventID'] = events_df['eventID'] + first_eventID
events_df = events_df.set_index('eventID')

print('finding and fixing baserunning...')
events_df = events_df.rename(columns={'theplay': 'play_orig'})
events_df = events_df.assign(theplay = events_df['play_orig'].apply(lambda x: x.split('.')[0]))
events_df = events_df.assign(baserunning_orig = events_df[events_df['play_orig'].apply(lambda x: len(x.split('.')) > 1)]['play_orig'].apply(lambda x: x.split('.')[1]))
events_df = events_df.assign(baserunning = events_df['baserunning_orig'].dropna().apply(lambda x: re.sub(nr_ur_rbi_fixer_re, '', x).replace('B', '0').replace('H', '4')))
events_df.loc[:, 'baserunning'] = events_df['baserunning'].fillna('')

# print('\tfixing pitch counts so they to be properly zero-padded when needed...')
# try:
#     events_df.loc[:, 'pitch_count'] = events_df['pitch_count'].apply(lambda x: '{0:02d}'.format(x))
# except ValueError:
#     print("\t\tCAN'T GET PITCH COUNT TO BE AN INTEGER FOR SOME REASON?...")
#     events_df.loc[:, 'pitch_count'] = pandas.to_numeric(events_df['pitch_count'], errors='coerce')

# print('splitting play into play description and baserunning...')
# events_df = events_df.rename(columns={'theplay': 'play_orig'})
# events_df = events_df.assign(baserunning = events_df[(events_df['play_orig'].apply(lambda x: '.' in x))]['play_orig'].apply(lambda x: x[x.find('.')+1:]))
# events_df = events_df.assign(play_pre_baserunning = events_df['play_orig'].apply(lambda x: x.split('.')[0]))
# events_df = events_df.assign(play_description = events_df[(events_df['play_pre_baserunning'].apply(lambda x: '/' in x))]['play_pre_baserunning'].apply(lambda x: x[x.find('/')+1:]))
# events_df = events_df.assign(theplay = events_df['play_pre_baserunning'].apply(lambda x: x.split('/')[0]))
# events_df = events_df.drop('play_pre_baserunning', axis=1)

# print('cleaning baserunning info...')
# events_df = events_df.assign(baserunning_cleaned = events_df['baserunning'].dropna().apply(lambda x: x.replace('B','0').replace('H', '4')))

# print('backing up...')
# events_df_bk = events_df

# games_df_bk = games_df
# rosters_df_bk = rosters_df
# events_df_bk = events_df
# new_things_df_bk = new_things_df
first_event_index = events_df.index.min()
last_event_index = events_df.index.max()

e = time.time()
g += e-s
print('\n')
print('Read {0:,.0f} games and {1:,.0f} roster entries and {2:,.0f} events in {3:.1f} seconds!'.format(len(games_df), len(rosters_df), len(events_df) ,e-s))
# # crazy game: 
# #games_df.loc['BRO191508312'][['visitor_score', 'home_score']]

# #events_df.sample(1).T

# # # #games_df.loc['BLF191504100'][['visitor', 'visitor_score', 'visitor_linescore', 'home', 'home_score', 'home_linescore']]
# # #games_df[['visitor_linescore', 'home_linescore']][games_df['visitor_linescore'] == '000000001']

#events_df.sample(1).T

Adding game context data for games in 1919...


reading rosters...
reading games...
	fixing linescores...
	checking for DH...
	getting fielder columns...
selecting games and roster rows for 1919...
Setting first eventID to be 280,586...
finding and fixing baserunning...


Read 1,118 games and 520 roster entries and 51,695 events in 12.8 seconds!


In [29]:
# #baserunning_fielder_error_re = re.compile('(\d|B|H)\(\d*(E|PB)\d*\)')  # had to add the possibility of (NR)(*E)
baserunning_fielder_error_re = re.compile('(?<!\))\(\d*E\d*\)')

def noplay(thisrow, nextrow, prevrow, state, printme = False):
    
    mystery_noplay_dict = {'BOS191508090': [35], 'BRO191508190': [70]}
    
    explained = False
    if (type(thisrow['comments']) == str):
        if (printme):
            print('\tEvent {0:0d} is a comment: {1:}'.format(thisrow['event_in_game'], thisrow['comments']))
        explained = True
    if (type(thisrow['pinch_runner_playerID']) == str):
        if (printme):
            print('\tEvent {0:0d}: pinch runner: {1:} replaces {2:}'.format(thisrow['event_in_game'], thisrow['pinch_runner_playerID'], thisrow['pinch_runner_replaced_playerID']))
        explained = True
        
    if (thisrow['event_in_game'] < nextrow['event_in_game']):  # prevents rolling over to event zero in next game
        if ((thisrow['batterID'] != nextrow['batterID']) | (thisrow['batterID'] != prevrow['batterID'])):
            if (printme):
                print('\tEvent {0:0d}: {1:} replaces {2:} at bat...'.format(thisrow['event_in_game'], nextrow['batterID'], thisrow['batterID']))
            explained = True
        if ((thisrow['pitcherID'] != nextrow['pitcherID']) | (thisrow['pitcherID'] != prevrow['pitcherID'])):
            if (printme):
                print('\tEvent {0:0d}: {1:} replaces {2:} as pitcher...'.format(thisrow['event_in_game'], nextrow['pitcherID'], thisrow['pitcherID']))
            explained = True
        for this_fielder in fielder_columns:   # hopefully fielder_columns is global?
            if ((thisrow[this_fielder] != nextrow[this_fielder]) | (thisrow[this_fielder] != prevrow[this_fielder])):
                if (printme):
                    print('\tEvent {0:0d}: {1:} replaces {2:} at {3:}...'.format(thisrow['event_in_game'], nextrow[this_fielder], thisrow[this_fielder], this_fielder))
                explained = True
#     if (courtesy_runner_half_inning != vh):
#         print('\tEvent {0:0d}: courtesy runner...'.format(thisrow['event_in_game']))
#         explained = True
    if (not(explained)):
        if (thisrow['gameID'] in list(mystery_noplay_dict.keys())):
            if (thisrow['event_in_game'] in mystery_noplay_dict[thisrow['gameID']]):
                print('\tMystery no play in game {0:} at event-in-game {1:.0f}!'.format(thisrow['gameID'], thisrow['event_in_game']))
                explained = True
        else:
            print('\tEvent {0:0d} (ix={1:0d}): UNEXPLAINED NO PLAY'.format(thisrow['event_in_game'], ix))
            print('This row:')
            print(thisrow)
            print('-------------------')
            print('\n')
            state['killswitch'] = True
    
    return state

def advance_runner(start_base, end_base, state, printme):
#     if (printme):
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
#         print('\t\t\tAdvancing runner from {0:} to {1:}!'.format(ordinaltg(start_base), ordinaltg(end_base)))
#         print('\t\t\t\tBases were: {0:}'.format(state['onbase']))
    if (end_base == 4):
        state['score'][vh] += 1
    else:
        if (start_base == 0):
            state['onbase'][end_base] = state['batterID']
            state['manual_batter_advance'] = True
        else:
            state['onbase'][end_base] = state['onbase'][start_base]
    if (start_base == 0):
        state['manual_batter_advance'] = True
    elif (start_base == end_base): # don't clear base if runner ended at same base
        pass
    else:
        state['onbase'][start_base] = ''
#     if (printme):
#         print('\t\t\t\tBases are now: {0:}'.format(state['onbase']))    
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
        
    return state

def runner_out(start_base, end_base, state, printme):
#     if (printme):
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
#         pprint(state)
#         print('\n')
#         print('\t\t\t Runner coming from {0:} is out at {1:}!'.format(ordinaltg(start_base), ordinaltg(end_base)))
#         print('\t\t\t\tBases were: {0:}'.format(state['onbase']))
    state['nOuts'] += 1
    if (end_base < 4):
        state['onbase'][end_base] = ''
    if (start_base == 0):
        state['manual_batter_advance'] = True
    else:
        state['onbase'][start_base] = ''
        
#     if (printme):
#         print('\n')
#         print('\t\t\t\tBases are now: {0:}'.format(state['onbase']))
#         print('\t\t\t^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')

    return state

def parse_baserunning(baserunning_str, printme = False):
    
    baseruns = []
    
    baserunning_str_list = sorted(baserunning_str.split(';'), reverse=True)
#     if (printme):
#         print('\tSTATE OF BASES BEFORE: {0:}'.format(state['onbase']))
#         print('\tBaserunning: {0:} | {1:}'.format(baserunning_str, baserunning_str_list))
    
    for this_baserun_str in baserunning_str_list:
        this_baserun = {}
        
        this_baserun['start_base'] = 0 if this_baserun_str[0] == 'B' else int(this_baserun_str[0])
        
        this_baserun['success'] = True if this_baserun_str[1] == '-' else False
        if (re.search(baserunning_fielder_error_re, this_baserun_str) != None):
            this_baserun['success'] = True

        this_baserun['end_base'] = 4 if this_baserun_str[2] == 'H' else int(this_baserun_str[2])

        baseruns.append(this_baserun)
        
    baseruns = sorted(baseruns, key=lambda d: d['start_base'], reverse=True)
    
#     if (printme):
#         print('`````````````````````')
#         print(baseruns)
#         print('`````````````````````')
    
    return baseruns

def baserunning(baserunning_str, state, printme = False):
    baseruns = parse_baserunning(baserunning_str, printme)
#     if (printme):
#         print(baseruns)
    for this_baserun in baseruns:
#         if (printme):
#             print('/////////////////////')
#             print(this_baserun)
        if (this_baserun['success']):
            state = advance_runner(this_baserun['start_base'], this_baserun['end_base'], state, printme)
        else:
            state = runner_out(this_baserun['start_base'], this_baserun['end_base'], state, printme)
#             if (printme):
# #                 print('\n')
# #                 pprint(state)
#                 print('/////////////////////')
    return state


def steal_bases(stealstring, state, printme = False, baserunning_str = ''):
    if (printme):
        print('\tggggggggggggggggggggg')
        print('\t\tstate of bases after this steal: {0:}'.format(state['onbase']))
        
    steal_list = sorted(stealstring.split(';'), reverse=True)

    if (printme):
        print('\tBases: {0:} | {1:}'.format(steal_list, stealstring))
    
    baseruns = []
    if (baserunning_str != ''):
        baseruns = parse_baserunning(baserunning_str, printme)
        if (printme):
            print('\t\tBaseruns: {0:} |{1:}'.format(baserunning_str, baseruns))
    
    highest_base_stolen = np.max([int(x[2].replace('B','0').replace('H','4')) for x in steal_list])
    lowest_base_stolen = np.min([int(x[2].replace('B','0').replace('H','4')) for x in steal_list])
    if (printme):
        print('Highest stolen base: {0:}, lowest stolen base: {1:}'.format(ordinaltg(highest_base_stolen), ordinaltg(lowest_base_stolen)))
    
    baseruns_beyond = [x for x in baseruns if x['start_base'] > highest_base_stolen]
    if ((printme) & (len(baseruns_beyond) > 0)):
        print('Baserunning beyond highest stolen base of {0:}: {1:}'.format(ordinaltg(highest_base_stolen), baseruns_beyond))
    for x in baseruns_beyond: 
        if (x['success']):
            state = advance_runner(x['start_base'], x['end_base'], state, printme)
        else:
            state = runner_out(x['start_base'], x['end_base'], state, printme)

    for this_steal in steal_list:
        base_stolen = int(this_steal[2].replace('B','0').replace('H','4'))
        source = -3
        for b in range(base_stolen-1, 0, -1):
            if (state['onbase'][b] != ''):
                source = b
                break
        if (source == -3): # if all prior bases are empty, batter steals
            source = 0
        if (printme):
            print('{0:} base is stolen |{1:}|'.format(ordinaltg(base_stolen), this_steal))
            print('Source was {0:} (onbase = {1:})'.format(ordinaltg(source), state['onbase']))
        
        baseruns_here = [x for x in baseruns if x['start_base'] == source]
        if (len(baseruns_here) > 0):
            if (printme):
                print('WE HAVE EXPLICIT BASERUNNING for the runner who started on {0:}:'.format(ordinaltg(source)))
            for this_explicit_baserun in baseruns_here:
                if (printme):
                    print('\t\t<<<<<{0:}>>>>>'.format(this_explicit_baserun))
                if (this_explicit_baserun['success']):
                    state = advance_runner(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
                else:
                    state = runner_out(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
        else:
            state = advance_runner(source, base_stolen, state, printme)
    
    baseruns_before = [x for x in baseruns if x['start_base'] < lowest_base_stolen-1]
    if ((printme) & (len(baseruns_before) > 0)):
        print('Baserunning prior to lowest stolen base of {0:}: {1:}'.format(ordinaltg(lowest_base_stolen), baseruns_before))
    for x in baseruns_before: 
        if (x['success']):
            state = advance_runner(x['start_base'], x['end_base'], state, printme)
        else:
            state = runner_out(x['start_base'], x['end_base'], state, printme)
            
    if (printme):
        print('\t\tstate of bases after this steal: {0:}'.format(state['onbase']))
        print('\tggggggggggggggggggggg')
    return state

def catch_stealing(catchstring, state, printme = False, baserunning_str = ''):
#     if (printme):
#         print('\thhhhhhhhhhhhhhhhhhh')
#         print('\tEvent: {0:}'.format(catchstring))
    caught_list = sorted(catchstring.split(';'), reverse=True)

#     if (printme):
#         print('\Caught stealing list: {0:} | {1:}'.format(caught_list, catchstring))
    
    baseruns = []
    if (baserunning_str != ''):
        baseruns = parse_baserunning(baserunning_str, printme)
#         if (printme):
#             print('\t\tBaseruns: {0:} |{1:}'.format(baserunning_str, baseruns))
    
    highest_base_caught_at = np.max([int(x[2].replace('B','0').replace('H','4')) for x in caught_list])
    lowest_base_caught_at = np.min([int(x[2].replace('B','0').replace('H','4')) for x in caught_list])
    
    baseruns_beyond = [x for x in baseruns if x['start_base'] > highest_base_caught_at-1]
    if (len(baseruns_beyond) > 0):
#         if (printme):
#             print('Baserunning from at or beyond highest base caught at of {0:}: {1:}'.format(ordinaltg(highest_base_caught_at), baseruns_beyond))
        for x in baseruns_beyond: 
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)

    for this_caught in caught_list:
        caught_at = int(this_caught[2].replace('B','0').replace('H','4'))

        source = -3
        for b in range(caught_at-1, 0, -1):
            if (state['onbase'][b] != ''):
                source = b
                break
        if (source == -3): # if all prior bases are empty, batter steals
            source = 0

        baseruns_here = [x for x in baseruns if x['start_base'] == source]
        if (len(baseruns_here) > 0):
#             if (printme):
#                 print('WE HAVE EXPLICIT BASERUNNING for the runner who started on {0:}:'.format(ordinaltg(source)))
            for this_explicit_baserun in baseruns_here:
#                 if (printme):
#                     print('\t\t<<<<<{0:}>>>>>'.format(this_explicit_baserun))
                if (this_explicit_baserun['success']):
                    state = advance_runner(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
                else:
                    state = runner_out(this_explicit_baserun['start_base'], this_explicit_baserun['end_base'], state, printme)
        else:
            if (re.search(baserunning_fielder_error_re, this_caught) != None):
#                 if (printme):
#                     print('Failed caught stealing at {0:} from source at {1:} |{2:}|'.format(ordinaltg(caught_at), source, this_caught))
                state = advance_runner(source, caught_at, state, printme)
            else:
#                 if (printme):
#                     print('Runner was caught at {0:} from source at {1:} |{2:}|'.format(ordinaltg(caught_at), source, this_caught))
                state = runner_out(source, caught_at, state, printme)
    
    baseruns_before = [x for x in baseruns if x['start_base'] < lowest_base_caught_at-1]
#    if ((printme) & (len(baseruns_before) > 0)):
    if (len(baseruns_before) > 0):
#         if (printme):
#             print('Baserunning prior to lowest base caught at of {0:}: {1:}'.format(ordinaltg(lowest_base_caught_at), baseruns_before))
        for x in baseruns_before: 
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)
                        
#     if (printme):
#         print('\thhhhhhhhhhhhhhhhhhh')
    return state

def pickoff(pickoff_str, state, printme = True, baserunning_str = ''):
#     if (printme):
#         print('wfe')
    pickoff_list = sorted(pickoff_str.split(';'), reverse = True)

#     if (printme):
#         print('\t\tPickoffs: {0:} |{1:}'.format(pickoff_str, pickoff_list))
    
    baseruns = []
    if (baserunning_str != ''):
        baseruns = parse_baserunning(baserunning_str, printme)
#         if (printme):
#             print('\t\tBaseruns: {0:} |{1:}'.format(baserunning_str, baseruns))
    for this_pickoff in pickoff_list:
        base_picked_off_at = int(this_pickoff[2].replace('B', '0').replace('H', '4'))
#         if (printme):
#             print('\t\t\tPicked off at {0:.0f} |{1:}|'.format(base_picked_off_at, this_pickoff))
        baseruns_beyond = [x for x in baseruns if x['start_base'] > base_picked_off_at]
        for x in baseruns_beyond:
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)
        baseruns_here = [x for x in baseruns if x['start_base'] == base_picked_off_at]
        if (len(baseruns_here) > 0):  # explicit baserunning from pickoff base
            for x in baseruns_here:
                if (x['success']):
                    state = advance_runner(x['start_base'], x['end_base'], state, printme)
                else:
                    state = runner_out(x['start_base'], x['end_base'], state, printme)
        else: # no baserunning from pickoff base, eval pickoff only
            if (re.search(baserunning_fielder_error_re, this_pickoff)):
                state = advance_runner(base_picked_off_at, base_picked_off_at+1, state, printme)
            else:
                state = runner_out(base_picked_off_at, base_picked_off_at+1, state, printme)

        baseruns_before = [x for x in baseruns if x['start_base'] < base_picked_off_at]
        for x in baseruns_before:
            if (x['success']):
                state = advance_runner(x['start_base'], x['end_base'], state, printme)
            else:
                state = runner_out(x['start_base'], x['end_base'], state, printme)            
    return state

print('ok')



ok


In [44]:
def fix_special_case_event(gameID, eventnum, state, printme = False):
    print('\tFixing a special case in game number {0:.0f} ({1:}) at event-in-game {2:.0f}: {3:}'.format(this_game_number, thisrow['gameID'], thisrow['event_in_game'], thisrow['play_orig']))
    
    if ((gameID == 'CHA191505010') and (eventnum == 18)):    
        print('\t\tExplanation: weavb101 safe on a pickoff error but stays at 3rd\n')
        state['onbase'][3] = 'weavb101'
        state['score'][1] = 0
    elif ((gameID == 'NY1191509290') and (eventnum == 22)):    
        print('\t\tExplanation: batter herbf101 still out despite passed ball\n')
        state['onbase'][1] = ''
        state['nOuts'] += 1
    elif ((gameID == 'PHI191507140') and (eventnum == 43)):  
        print('\t\tExplanation: strikeout error was on baserunning, batter beckb101 still out\n')
        state['onbase'][1] = ''
        state['nOuts'] += 1
    elif ((gameID == 'PHI191509092') and (eventnum == 40)):  
        print('\t\tExplanation: batter paskd101 still out at 3rd despite fielding error\n')
        state['onbase'][3] = ''
        state['nOuts'] += 1
    elif ((gameID == 'BOS191609280') and (eventnum == 35)):  
        print('\t\tExplanation: fielding error on pickoff but mille102 stays at 3rd (and does not score)\n')
        state['onbase'][3] = 'mille102'
        state['score'][0] += -1
    elif ((gameID == 'PHA191605170') and (eventnum == 22)):  
        print('\t\tExplanation: double steal of 2nd and 3rd was recorded in wrong order\n')
        state['onbase'][3] = 'walsj108'
        state['onbase'][2] = 'pickc103'
    elif ((gameID == 'BRO191608040') and (eventnum == 16)):  
        print('\t\tExplanation: fielding error on pickoff but stenc101 stays at 3rd (and does not score)\n')
        state['onbase'][3] = 'stenc101'
        state['score'][1] += -1
    elif ((gameID == 'NY1191606210') and (eventnum == 8)):  
        print('\t\tExplanation: double play with baserunners, batter kaufb101 reaches 1st\n')
        state['nOuts'] += -1
    elif ((gameID == 'PIT191605130') and (eventnum == 57)):  
        print('\t\tExplanation: fielding error on pickoff but barne104 stays at 3rd (and does not score)\n')
        state['onbase'][3] = 'barne104'
        state['score'][1] += -1
    elif ((gameID == 'PHA191708111') and (eventnum == 37)):  
        print('\t\tExplanation: fielding error on pickoff but schaw101 stays at 3rd (and does not score)\n')
        state['onbase'][3] = 'schaw101'
        state['score'][1] += -1
    elif ((gameID == 'PHI191710022') and (eventnum == 25)):  
        print('\t\tExplanation: zimmh101 caught at home despite fielding error\n')
        state['score'][0] += -1
        state['nOuts'] += 1
    elif ((gameID == 'PIT191706090') and (eventnum == 3)):  
        print('\t\tExplanation: wilhj101 still out on passed ball, only runners advance\n')
        state['nOuts'] += 1
        state['onbase'][1] = ''
    elif ((gameID == 'PIT191706210') and (eventnum == 8)):  
        print('\t\tExplanation: fielding error on pickoff but mannl101 stays at 3rd (and does not score)\n')
        state['onbase'][3] = 'mannl101'
        state['score'][0] += -1
    elif ((gameID == 'CHA191806200') and (eventnum == 67)):  
        print('\t\tExplanation: colle101 still out at 3rd despite fielding error\n')
        state['nOuts'] += 1
        state['onbase'][3] = ''
    elif ((gameID == 'PHA191807120') and (eventnum == 56)):  
        print('\t\tExplanation: demmr101 still out on passed ball, only runners advance\n')
        state['nOuts'] += 1
    elif ((gameID == 'CHA191907100') and (eventnum == 52)):  
        print('\t\tExplanation: mcmuf101 scores on fielding error\n')
        state['nOuts'] += -1
        state['score'][1] += 1
    elif ((gameID == 'DET191906021') and (eventnum == 28)):  
        print('\t\tExplanation: felsh101 scores on fielding error\n')
        state['nOuts'] += -1
        state['score'][0] += 1

    return state

print('ok')

ok


In [46]:
s = time.time()
start_at_game_number = 165
pfe = -1
# 1918: 

skip100 = False
printme = False
courtesy_runner_half_inning = -1

non_batter_re = re.compile('^(SB|CS|PO|BK|WP|PB|OA)')
sb_re = re.compile('^(SB(\d|H);*)+')
caught_stealing_re = re.compile('CS(\d|H)')
#pickoff_re = re.compile('(PO\d\(*E*\d*\))+')
#pickoff_re = re.compile('(PO\d\(\d*E+\d*\))+')
pickoff_re = re.compile('(PO\d\;*)+')


hit_re = re.compile('^(S(?!B)|D|T|HR)(\d|GR)*')
onbase_nohit_re = re.compile('^(\d|FL|FC)*(W|IW|HP|C|E|FC)')
fielder_out_re = re.compile('^\d+.*')

base_updater_atom_re = re.compile('\((\d|B|H)\)')
#base_updater_atom_re = re.compile('(\d)+\((\d|B|H)\)')
mp_re = re.compile('(?<!N)(D|T)P')

print('creating a new dataframe to hold what we calculate here...')
new_things_df = pandas.DataFrame(data=None, columns=new_columns, index=events_df.index)
max_event_id = -1

# NOTE: we HAVE TO keep all the NPs because pitches occurred on all of them!
print('\nparsing plays...')

gamelist = events_df['gameID'].unique().tolist()
print('Found {0:,.0f} events from {1:,.0f} games-with-events!'.format(len(events_df), len(gamelist)))
print('\n')

for this_game_number in range(start_at_game_number, len(gamelist)):
    thisgameID = gamelist[this_game_number]
    if (skip100):
        if (np.mod(this_game_number, 100) == 0):
            print('Initializing game {0:.0f} of {1:.0f}: {2:}...'.format(this_game_number, len(gamelist), thisgameID))
    else:
        print('Initializing game {0:.0f} of {1:.0f}: {2:}'.format(this_game_number, len(gamelist), thisgameID))
    #print(games_df.loc[gamelist[this_game_number]][['visitor', 'home']])
#    print('\n')
    state = {}
    state['score'] = [0,0]
    state['nOuts'] = 0
    state['onbase'] = {1: '', 2: '', 3: ''}
    state['manual_batter_advance'] = False
    state['batterID'] = ''
    state['vh'] = -1
    state['killswitch'] = False
    already_said_ended_early = False

    this_game_innings = events_df[events_df['gameID'] == thisgameID]['inning'].unique()              
    for this_inning in this_game_innings:
        for vh in [0,1]:
            state['vh'] = vh
            state['nOuts'] = 0
            for i in range(1,3+1):
                state['onbase'][i] = ''
            if (vh == 0):
                inningside = 'top'
            else:
                inningside = 'bottom'
            if (printme):
                print('====================================================')
                print('  {0:} of the {1:} has {2:.0f} plays...'.format(inningside.title(), ordinaltg(this_inning), len(events_df[(events_df['gameID'] == thisgameID) & (events_df['inning'] == this_inning) & (events_df['visitor_or_home'] == vh)])))
                print('====================================================')
            for ix, thisrow in events_df[
                    (events_df['gameID'] == thisgameID) 
                    & (events_df['inning'] == this_inning) 
                    & (events_df['visitor_or_home'] == vh)
            ].sort_values(by='event_in_game').iterrows():                
                new_things_df.loc[ix, ['vis_score', 'home_score', 'outs', 'runner_1b', 'runner_2b','runner_3b']
                                 ] = [state['score'][0], state['score'][1], state['nOuts'], state['onbase'][1], state['onbase'][2], state['onbase'][3]]
                
                state['batterID'] = thisrow['batterID']
                state['manual_batter_advance'] = False
                
                if (thisrow['event_in_game'] >= pfe):
                    printme = True
                    print_batterID_at_end = False
                if (printme):
                    print('     ***')
                    print('     Event {0:.0f} starting state: V {1:.0f} H {2:.0f}, {3:.0f} outs, onbase = {4:}, {5:} at bat'.format(thisrow['event_in_game'], state['score'][0], state['score'][1], state['nOuts'], state['onbase'], thisrow['batterID']))
                if (printme):
                    print('\tEvent {0:.0f}: {1:}'.format(thisrow['event_in_game'], thisrow['play_orig']))

                has_baserunning = False
            
                if (thisrow['baserunning'] != ''):
                    has_baserunning = True

                if (thisrow['theplay'][0:2] == 'NP'):
                    print_batterID_at_end = True
                    if (ix == last_event_index):
                        noplay(thisrow, thisrow, events_df.loc[ix-1], state, printme)
                    elif (ix == first_event_index):
                        noplay(thisrow, events_df.loc[ix+1], thisrow, state, printme)
                    else:
                        noplay(thisrow, events_df.loc[ix+1], events_df.loc[ix-1], state, printme)
                elif (re.search(non_batter_re, thisrow['theplay']) != None):
                    print_batterID_at_end = True
                    thematch = re.match(non_batter_re, thisrow['theplay'])
                    if (printme):
                        print('Non batter event: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                    if (re.search(sb_re, thisrow['theplay']) != None):
                        sbmatch = re.match(sb_re, thisrow['theplay'])
                        if (printme):
                            print('Stolen base: {0:} |{1:}|'.format(sbmatch, thisrow['theplay']))
                        if (has_baserunning):
                            state = steal_bases(thisrow['theplay'], state, printme, thisrow['baserunning'])
                        else:
                            state = steal_bases(thisrow['theplay'], state, printme)
                    elif ('POCS' in thisrow['theplay']): 
                        if (printme):
                            print('Pickoff plus caught stealing: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                        if (has_baserunning):
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme, thisrow['baserunning'])
                        else:
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme)
                    elif ('CS' in thisrow['theplay']): 
                        if (printme):
                            print('Caught stealing: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                        if (has_baserunning):
                            state = catch_stealing(thisrow['theplay'], state, printme, thisrow['baserunning'])
                        else:
                            state = catch_stealing(thisrow['theplay'], state, printme)
                    elif (re.search(pickoff_re, thisrow['theplay']) != None):
                        pomatch = re.search(pickoff_re, thisrow['theplay'])
                        if (printme):
                            print('Pickoff: {0:} |{1:}|'.format(thisrow['theplay'][pomatch.start():pomatch.end()], thisrow['theplay']))
                        if (has_baserunning):
                            state = pickoff(thisrow['theplay'], state, printme, thisrow['baserunning'])
                        else:
                            state = pickoff(thisrow['theplay'], state, printme)
                        
                    ### Events that advance baserunners but never batter
                    elif (('BK' in thisrow['theplay']) or ('WP' in thisrow['theplay']) or ('PB' in thisrow['theplay']) or ('OA' in thisrow['theplay'])):
                        if (printme):
                            print('Event that advances baserunners but not batter: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                        state = baserunning(thisrow['baserunning'], state, printme)

                elif (re.search(hit_re, thisrow['theplay']) != None):   ### Next, events that get batter on base
                    thematch = re.match(hit_re, thisrow['theplay'])
                    if (printme):
                        print('\t\tHit: {0:} |{1:}|'.format(thematch, thisrow['theplay']))
                    if (has_baserunning):
                        state = baserunning(thisrow['baserunning'], state, printme)
                    if (thisrow['theplay'][0] == 'S'):
                        if not(state['manual_batter_advance']):
                            state['onbase'][1] = thisrow['batterID']
                    elif (thisrow['theplay'][0] == 'D'):
                        if not(state['manual_batter_advance']):
                            state['onbase'][2] = thisrow['batterID']
                    elif (thisrow['theplay'][0] == 'T'):
                        if not(state['manual_batter_advance']):
                            state['onbase'][3] = thisrow['batterID']
                    elif (thisrow['theplay'][0:2] == 'HR'):
                        if not(state['manual_batter_advance']):
                            state['score'][vh] += 1
                        
                elif (re.search(onbase_nohit_re, thisrow['theplay']) != None):   ### Next, other events that get batter on base
                    if (printme):
                        if ('+' in thisrow['theplay']):
                            print('\t\tEvent {0:.0f}: On base with no hit plus extra: {1:}'.format(thisrow['event_in_game'], thisrow['theplay']))
                        else:
                            print('\t\tOn base with no hit!')
#                     if ('+' in thisrow['theplay']):
#                         elif ('PO' in thisrow['theplay']):                            
#                             if (has_baserunning):
#                                 state = pickoff(thisrow['theplay'][thisrow['theplay'].find('PO'):], state, printme, thisrow['baserunning'])
#                             else:
#                                 state = pickoff(thisrow['theplay'][thisrow['theplay'].find('PO'):], state, printme, thisrow['baserunning'])
#                         elif ('E' in thisrow['theplay']):
#                             state = baserunning(thisrow['baserunning'], state, printme)
#                         elif ('WP' in thisrow['theplay']):
#                             state = baserunning(thisrow['baserunning'], state, printme)
#                         elif ('PB' in thisrow['theplay']):
#                             state = baserunning(thisrow['baserunning'], state, printme)
#                         else:
#                             print('+ event not found: {0:}'.format(thisrow['theplay']))
#                     else:
                    if ('W+SB' in thisrow['theplay']):
                        if (has_baserunning):
                            state = steal_bases(thisrow['theplay'][thisrow['theplay'].find('SB'):], state, printme, thisrow['baserunning'])
                        else:
                            state = steal_bases(thisrow['theplay'][thisrow['theplay'].find('SB'):], state, printme)
                    elif ('W+CS' in thisrow['theplay']):
                        if (has_baserunning):
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme, thisrow['baserunning'])
                        else:
                            state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme)
                    else:
                        if (has_baserunning):  # advance bases if they weren't advanced in a + event
                            state = baserunning(thisrow['baserunning'], state, printme)
                    if (not(state['manual_batter_advance'])):
                        state['onbase'][1] = state['batterID'] # regardless of events given in +, batter goes to 1st
                            
                elif (thisrow['theplay'][0] == 'K'): ### Start with events that get the batter out
                    if (printme):
                        if ('+' in thisrow['theplay']):
                            print('\t\tEvent {0:.0f}: Strikeout plus extra: {1:}'.format(thisrow['event_in_game'], thisrow['theplay']))
                        else:
                            print('\t\tStrikeout!')
                    if ('+' in thisrow['theplay']):
                        if ('SB' in thisrow['theplay']):
                            if (has_baserunning):
                                state = steal_bases(thisrow['theplay'][thisrow['theplay'].find('SB'):], state, printme, thisrow['baserunning'])
                            else:
                                state = steal_bases(thisrow['theplay'][thisrow['theplay'].find('SB'):], state, printme)
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1    # batter still out
                        elif ('CS' in thisrow['theplay']):
                            if (has_baserunning):
                                state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme, thisrow['baserunning'])
                            else:
                                state = catch_stealing(thisrow['theplay'][thisrow['theplay'].find('CS'):], state, printme)
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1    # batter still out
                        elif ('PO' in thisrow['theplay']):
                            if (has_baserunning):
                                state = pickoff(thisrow['theplay'][2:], state, printme, thisrow['baserunning'])
                            else:
                                state = pickoff(thisrow['theplay'][2:], state, printme)
                            if not(state['manual_batter_advance']):
                                state['nOuts'] += 1    # batter still out
                        elif ('E' in thisrow['theplay']):
                            state = baserunning(thisrow['baserunning'], state, printme)
                        elif ('WP' in thisrow['theplay']):
                            state = baserunning(thisrow['baserunning'], state, printme)
                            if (not(state['manual_batter_advance'])):
                                state['nOuts'] += 1    # batter still out
                        elif ('PB' in thisrow['theplay']):
                            state = baserunning(thisrow['baserunning'], state, printme)
                            if (not(state['manual_batter_advance'])): # batter still out, unlesss... explicit batter baserunning OR
                                if ('K23' in thisrow['theplay']):  # ...K23 (C to 1B after dropped 3rd strike)
                                    state['nOuts'] += 1
                                else:
                                    state['onbase'][1] = state['batterID']

                    else:
                        if (has_baserunning):
                            state = baserunning(thisrow['baserunning'], state, printme)
                        if (not(state['manual_batter_advance'])):
                            state['nOuts'] += 1
                    
#                     if (not(state['manual_batter_advance'])):
#                         state['nOuts'] += 1
                        
                elif (re.search(fielder_out_re, thisrow['theplay'])):
                    if (re.search(base_updater_atom_re, thisrow['theplay']) == None):
                        if (printme):
                            print('\t\tFielder out: {0:}'.format(thisrow['theplay']))
                        state['nOuts'] += 1
                        if (has_baserunning):
                            state = baserunning(thisrow['baserunning'], state, printme)
                    else:
                        if (printme):
                            print('\t\tFielder out with shenanigans: {0:}'.format(thisrow['theplay']))
                        involves_batter = False
                        batter_implied = False
                        base_updater_iter = re.finditer(base_updater_atom_re, thisrow['theplay'])
                        nBaseUpdaters = 0
                        for this_base_updater in base_updater_iter:
                            nBaseUpdaters += 1
                            from_base_str = thisrow['theplay'][this_base_updater.start()+1:this_base_updater.end()-1]
                            if (from_base_str == 'B'):
                                if (printme):
                                    print('\t{0:}: batter out'.format(this_base_updater))
                                involves_batter = True
                            else:
                                from_base = int(thisrow['theplay'][this_base_updater.start()+1:this_base_updater.end()-1])
                                if (printme):
                                    print('\t{0:}: runner from {1:} out'.format(this_base_updater, ordinaltg(from_base)))

                                state['onbase'][from_base] = ''
                                state['nOuts'] += 1
                        if (re.search(mp_re, thisrow['theplay']) != None):
                            mpmatch = re.search(mp_re, thisrow['theplay'])
                            if (printme):
                                print('MULTIPLE PLAY: {0:} |{1:}|, {2:.0f} base updaters'.format(thisrow['theplay'][mpmatch.start():mpmatch.end()], thisrow['theplay'], nBaseUpdaters))
                            if ((thisrow['theplay'][mpmatch.start():mpmatch.end()] == 'DP') and (nBaseUpdaters < 2)):
                                batter_implied = True
                            #state['killswitch'] = True
                        if (has_baserunning):
                            state = baserunning(thisrow['baserunning'], state, printme)
                        if (not(state['manual_batter_advance'])):
                            if (involves_batter or batter_implied):
                                state['nOuts'] += 1
                            else:
                                state['onbase'][1] = state['batterID']
                        if (state['nOuts'] > 3):
                            state['nOuts'] = 3
                else:
                    print('UNKNOWN EVENT: {0:}'.format(thisrow['theplay']))

                ### SPECIAL CASES!
                if (thisrow['gameID'] in list(special_case_events.keys())):
                    if (thisrow['event_in_game'] in special_case_events[thisrow['gameID']]):
                        state = fix_special_case_event(thisrow['gameID'], thisrow['event_in_game'], state)
                    
                if (printme):
                    if (print_batterID_at_end):
                        print('     Event {0:.0f} ending state: V {1:.0f} H {2:.0f}, {3:.0f} outs, onbase = {4:}, {5:} at bat'.format(thisrow['event_in_game'], state['score'][0], state['score'][1], state['nOuts'], state['onbase'], thisrow['batterID']))
                    else:
                        print('     Event {0:.0f} ending state: V {1:.0f} H {2:.0f}, {3:.0f} outs, onbase = {4:}'.format(thisrow['event_in_game'], state['score'][0], state['score'][1], state['nOuts'], state['onbase']))
                    print('     ***')
                    print('\n')
            # checks: each half-inning must end with 3 outs, and each inning must end with scores matching game log
            if (state['nOuts'] != 3):
                if (thisrow['gameID'] in early_end_game_list):
                    if (not(already_said_ended_early)):
                        print('\tGAME NUMBER {0:.0f} ENDED EARLY: {1:}'.format(this_game_number, thisrow['gameID']))
                        print('\n')
                        already_said_ended_early = True
                else:
                    if not((this_inning == np.max(this_game_innings)) & (vh == 1)):
                        print('DISCREPANCY: {0:} of the {1:} has {2:0d} outs!'.format(inningside.title(), ordinaltg(this_inning), state['nOuts']))
                        state['killswitch'] = True
                        max_event_id = ix
            skips = 0
            sco = 0
            if (games_df.loc[thisgameID]['has_linescores']):
                if (vh == 0):
                    for j in range(0, this_inning):
                        if (games_df.loc[thisgameID]['visitor_linescore'][j] == '('):
                            visitor_linescore_raw = games_df.loc[thisgameID]['visitor_linescore']
                            skips = visitor_linescore_raw[j:].find(')')
                            sco += int(visitor_linescore_raw[visitor_linescore_raw.find('(')+1:visitor_linescore_raw.find(')')])
                        else:
                            sco += int(games_df.loc[thisgameID]['visitor_linescore'][j+skips])
                    if (state['score'][0] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['visitor'], state['score'][0], sco, games_df.loc[thisgameID]['visitor_linescore']))
                        state['killswitch'] = True
                        max_event_id = ix
                else:
                    for j in range(0, this_inning):
                        if (games_df.loc[thisgameID]['home_linescore'][j+skips] != 'x'):
                            if (games_df.loc[thisgameID]['home_linescore'][j] == '('):
                                home_linescore_raw = games_df.loc[thisgameID]['home_linescore']
                                skips = home_linescore_raw[j:].find(')')
                                sco += int(home_linescore_raw[home_linescore_raw.find('(')+1:home_linescore_raw.find(')')])
                            else:
                                sco += int(games_df.loc[thisgameID]['home_linescore'][j+skips])
                    if (state['score'][1] != sco):
                        print('SCORE DISCREPANCY: {0:} of the {1:}: {2:} has {3:0d} and they should have {4:0d} <linescore: {5:}>'.format(inningside, ordinaltg(this_inning), games_df.loc[thisgameID]['home'], state['score'][1], sco, games_df.loc[thisgameID]['home_linescore']))
                        state['killswitch'] = True
                        max_event_id = ix            
            if (state['killswitch']):
                break
        if (state['killswitch']):
            break
    if (state['killswitch']):
        break
    else:
        max_event_id = ix

events_contexts_df = pandas.concat((events_df.loc[events_df.index <= max_event_id], new_things_df.loc[new_things_df.index <= max_event_id]), axis=1)
e = time.time()
g += e-s
print('\n')

print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(events_contexts_df), np.floor((e-s)/60), (e-s)%60))




creating a new dataframe to hold what we calculate here...

parsing plays...
Found 51,695 events from 628 games-with-events!


Initializing game 165 of 628: PHA191908040
     ***
     Event 1 starting state: V 0 H 0, 0 outs, onbase = {1: '', 2: '', 3: ''}, austj101 at bat
	Event 1: W
		On base with no hit!
     Event 1 ending state: V 0 H 0, 0 outs, onbase = {1: 'austj101', 2: '', 3: ''}
     ***


     ***
     Event 2 starting state: V 0 H 0, 0 outs, onbase = {1: 'austj101', 2: '', 3: ''}, gedej101 at bat
	Event 2: 53.1-2
		Fielder out: 53
     Event 2 ending state: V 0 H 0, 1 outs, onbase = {1: '', 2: 'austj101', 3: ''}
     ***


     ***
     Event 3 starting state: V 0 H 0, 1 outs, onbase = {1: '', 2: 'austj101', 3: ''}, jacow101 at bat
	Event 3: 63.2-3
		Fielder out: 63
     Event 3 ending state: V 0 H 0, 2 outs, onbase = {1: '', 2: '', 3: 'austj101'}
     ***


     ***
     Event 4 starting state: V 0 H 0, 2 outs, onbase = {1: '', 2: '', 3: 'austj101'}, sislg101 at bat
	Event 

In [26]:
s = time.time()
events_contexts_df = pandas.concat((events_df, new_things_df), axis=1)
#new_things_df
events_contexts_df.columns.tolist()

events_contexts_df = events_contexts_df[final_column_order]
events_contexts_df.to_csv('{0:0d}.csv'.format(thisyear))
e = time.time()
g += e-s
print('Processed {0:,.0f} events in {1:.0f} minutes {2:.0f} seconds!'.format(len(new_things_df), np.floor(g/60), g%60))


Processed 51,891 events in 2 minutes 40 seconds!


In [ ]:

# events_df[
#     (events_df['gameID'] == 'DET191808010') 
#     & ((events_df['event_in_game'] > 71) & (events_df['event_in_game'] < 76)
#       )
# ][['play_orig', 'field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID']]

# #['gameID', 'event_in_game', 'batterID', 'pitcherID', 'badj', 'ladj', 'pinch_runner_playerID', 'play_orig', 'comments']]
# #[['field_C_playerID', 'field_1B_playerID', 'field_2B_playerID', 'field_3B_playerID', 'field_SS_playerID', 'field_LF_playerID', 'field_CF_playerID', 'field_RF_playerID']]
